In [6]:
import json
import os
from library_dicom.dicom_processor.model.Series import Series 
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.csv_reader.MaskBuilder import MaskBuilder

from library_dicom.dicom_processor.tools.folders import *
from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *

import numpy as np
import SimpleITK as sitk
import csv

#### - Get dataset from  PARSE_FILTERED_DICOM_DATASET script

In [7]:
#LIST_PATH from json
json_path = '/media/deeplearning/Elements/REMARC_NIFTI/false_mask_serie.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)


In [8]:
number_of_study = len(dataset)
print("number of study : ", number_of_study)

number of study :  82


#### - Add CSV file for every study on the dataset

In [ ]:
#add csv_file from csv 

csv_path= '/home/deeplearning/AHL/AHL_CSV_DATA/pet0.csv'
csv_file = []
with open(csv_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ';') #liste pour chaque ligne 
    csv_file = []
    for row in reader :
        csv_file.append(row)
        
del csv_file[0] #enlever première ligne

In [4]:
#add csv_file from a list 

csv_path = '/media/deeplearning/Elements/REMARC_csv'
liste_csv_file = os.listdir(csv_path)
liste_csv = []
for csv_file in liste_csv_file : 
    liste_csv.append(os.path.join(csv_path, csv_file))

for data in dataset : 
    for csv_f in liste_csv : 
        if data[-1] in csv_f : 
            data.append(csv_f)

#### - Verify if every study has a csv file, 
####            If not, write a json , remove study from dataset

In [ ]:
check_csv = []
for data in dataset : 
    if len(data) != 8 : 
        check_csv.append(data)

print(len(check_csv))
write_json_file('/media/deeplearning/Elements/REMARC_Validated_DICOMS', 'check_csv', check_csv)

for r in check_csv : 
    dataset.remove(r)

#### -  Rewrite path and save complete dataset as json file 

In [7]:
#rewrite path 

for data in dataset : 
    new_1 = data[0].replace('/media/oncopole/DD 2To', '/media/deeplearning/Elements')
    new_2 = data[2].replace('/media/oncopole/DD 2To', '/media/deeplearning/Elements')
    data[0] = new_1
    data[2] = new_2

In [8]:
print('Nombre de study : ', len(dataset))

Nombre de study :  182


In [9]:
write_json_file('/media/deeplearning/Elements/REMARC_Validated_DICOMS', 'validated_dataset', dataset)

#### - Remove remove_bi_file if it exists in every serie 

In [ ]:
from library_dicom.dicom_processor.tools.folders import *
directory_validated_dicom = ''
paths = get_series_path(directory_validated_dicom)

for path in paths : 
    remove_bi_file(path)

### Different scripts : 

#### - Check if there is Unconstance Spacing error in every study of the dataset 

In [ ]:
unconstant_spacing = []
serie_error = []
for serie in dataset : 
    print(dataset.index(serie))
    try : 
        if serie[1] =='PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            if serie_pt_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing PET")
                unconstant_spacing.append(serie)
            
            serie_ct_objet = SeriesCT(serie[2])
            nifti_array_ct = serie_ct_objet.get_numpy_array()
            if serie_ct_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing CT")
                unconstant_spacing.append(serie)

        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            if serie_pt_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing PT")
                unconstant_spacing.append(serie)
            serie_ct_objet = SeriesCT(serie[0])
            nifti_array_ct = serie_ct_objet.get_numpy_array()
            if serie_ct_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing CT")
                unconstant_spacing.append(serie)

    except Exception as err : 
        print(serie)
        print(err)
        serie_error.append(serie)


#### - Generate PET, CT, MASK NIFTI with checking suv_max, mean and sd

In [ ]:
#Generate PET, CT, MASK NIFTI with checking suv_max, mean and sd

nifti_directory = '/media/deeplearning/Elements/REMARC_NIFTI'
mip_directory = '/media/deeplearning/Elements/REMARC_MIP'

#save serie_path with false mask 
serie_false_mask = []
#save result about serie with false mask 
results_false_mask = []
#save path of MIP to generate PDF 
path_mip = []

#save error serie 
error_dataset = []

for serie in dataset: 
    try : 
        print(dataset.index(serie))
        subliste = []
        if serie[1] == 'PT' : 
            
            serie_pt_objet = SeriesPT(serie[0])
            
            serie_pt_objet.get_instances_ordered()
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]
            threshold = mask_objet.details_rois['SUVlo']

            if mask_objet.is_correct_suv(nifti_array) == True : 

                print("MASK CORRECT")
                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")

                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[2])
                serie_ct_objet.get_instances_ordered()
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")

                #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                print("EXPORT NIFTI MASK")


            else : 
                results = []
                print("FALSE MASK")
                serie_false_mask.append(serie)

                print(mask_objet.calcul_suv(nifti_array))
                results.append(mask_objet.calcul_suv(nifti_array))

                print(mask_objet.ecart_suv_max(nifti_array))
                results.append(mask_objet.ecart_suv_max(nifti_array))

                print(mask_objet.ecart_suv_mean(nifti_array))
                results.append(mask_objet.ecart_suv_mean(nifti_array))

                print(mask_objet.ecart_SD(nifti_array))
                results.append(mask_objet.ecart_SD(nifti_array))

                results_false_mask.append(results)

                #threshold mask 41% 
                mask_4D = threshold_matrix(mask_4D, nifti_array, 0.41)

                #create mip for false mask and check 
                #pet
                angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, 'pet', cmap='Greys', vmin=0, vmax=7) 
                subliste.append(angle_filename)
                print('MIP PET')
               # mask
                angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, cmap='Greys')
                subliste.append(angle_filename_mask)
                print('MIP MASK')
                path_mip.append(subliste)


        else : 
            serie_pt_objet = SeriesPT(serie[2])
            serie_pt_objet.get_instances_ordered()
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]
            threshold = mask_objet.details_rois['SUVlo']

            if mask_objet.is_correct_suv(nifti_array) == True :  
                print("MASK CORRECT")
                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")

                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[0])
                serie_ct_objet.get_instances_ordered()
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")

                #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                print("EXPORT NIFTI MASK")



            else : 
                results = []
                print("FALSE MASK")
                serie_false_mask.append(serie)

                print(mask_objet.calcul_suv(nifti_array))
                results.append(mask_objet.calcul_suv(nifti_array))

                print(mask_objet.ecart_suv_max(nifti_array))
                results.append(mask_objet.ecart_suv_max(nifti_array))

                print(mask_objet.ecart_suv_mean(nifti_array))
                results.append(mask_objet.ecart_suv_mean(nifti_array))

                print(mask_objet.ecart_SD(nifti_array))
                results.append(mask_objet.ecart_SD(nifti_array))

                results_false_mask.append(results)

                #threshold mask 41% 
                mask_4D = threshold_matrix(mask_4D, nifti_array, 0.41)

                #create mip for false mask and check 
                #pet
                angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, 'pet', cmap='Greys', vmin=0, vmax=7) 
                subliste.append(angle_filename)
                print("MIP PET")
                #mask
                angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, cmap='Greys')
                subliste.append(angle_filename_mask)
                print('MIP MASK ')
                path_mip.append(subliste)
               

    except Exception as err : 
        print(serie)
        print(err)
        error_dataset.append(serie)

In [ ]:
print("number of error : ", len(error_dataset))
print("number of false mask : " , len(serie_false_mask))

In [ ]:
#Save different json 
directory_nifti = '/media/deeplearning/Elements/REMARC_NIFTI'
write_json_file(directory_nifti, 'false_mask_serie', serie_false_mask)
write_json_file(directory_nifti, 'false_mask_results', results_false_mask)
write_json_file(directory_nifti, 'error', error_dataset)


#### - Generate PET, CT, MASK NIFTI without checking

In [9]:
nifti_directory ='/media/deeplearning/Elements/REMARC_NIFTI'

error_dataset = []

for serie in dataset: 
    try : 
        print(dataset.index(serie))

        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            serie_pt_objet.get_instances_ordered()
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]

            print("MASK CORRECT")
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")

                    #generation nifti CT
            serie_ct_objet = SeriesCT(serie[2])
            serie_ct_objet.get_instances_ordered()
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")

                    #generation nifti mask
            filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
            print("EXPORT NIFTI MASK")
          
          
        else : 
            serie_pt_objet = SeriesPT(serie[2])
            serie_pt_objet.get_instances_ordered()
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(serie[-1], size_matrix)
            mask_4D = mask_objet.mask_array
            number_roi = mask_4D.shape[3]

            print("MASK CORRECT")
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")

                     #generation nifti CT
            serie_ct_objet = SeriesCT(serie[0])
            serie_ct_objet.get_instances_ordered()
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")

                    #generation nifti mask
            filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
            print("EXPORT NIFTI MASK")
           
    except Exception as err : 
        print(err)
        print(serie)
        error_dataset.append(serie)

a/deeplearning/Elements/REMARC_Validated_DICOMS/11009101436019/pet0/1.2.840.113704.1.111.3984.1354871612.14/1.2.840.113704.1.111.3984.1354871612.14', 'CT', '1.2.124.113532.172.16.25.2.20121204.748.1195904', 'V0', '11009101436019', '/media/deeplearning/Elements/REMARC_csv/11009101436019_dec 7_2012.csv']
48
'SeriesPT' object has no attribute 'instance_array'
['/media/deeplearning/Elements/REMARC_Validated_DICOMS/11009101226026/pet0/1.2.840.113704.1.111.4252.1353485746.8/1.2.840.113704.1.111.4252.1353485746.8', 'CT', '/media/deeplearning/Elements/REMARC_Validated_DICOMS/11009101226026/pet0/1.3.46.670589.28.2.12.21.6106.64577.2.1468.0.1353487372/1.3.46.670589.28.2.12.21.6106.64577.2.1468.0.1353487372', 'PT', '2.16.840.1.113669.632.20.210791.20000032746', 'V0', '11009101226026', '/media/deeplearning/Elements/REMARC_csv/11009101226026_nov 21_2012.csv']
49
'SeriesPT' object has no attribute 'instance_array'
['/media/deeplearning/Elements/REMARC_Validated_DICOMS/11009103076002/pet0/1.3.46.6705

In [ ]:
print("number of error : ", len(error_dataset))

#### - Generate PET & CT NIFTI only

In [ ]:
nifti_directory = '/home/deeplearning/AHL/AHL_NIFTI/pet4'

error = []
for serie in pet_4 : 
    print(pet_4.index(serie))
    try : 
        if serie[1] == 'PT' : 
                serie_pt_objet = SeriesPT(serie[0]) 
                serie_pt_objet.get_instances_ordered()
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

                                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[2])
                serie_ct_objet.get_instances_ordered()
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")

        else : 
                serie_pt_objet = SeriesPT(serie[2])
                serie_pt_objet.get_instances_ordered() 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

                        
                                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[0])
                serie_ct_objet.get_instances_ordered()
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
 

    except Exception as err : 
        print(serie)
        print(err)
        error.append(serie)